## Thêm một biến để nắm bắt NA

Trong các notebook trước, chúng ta đã học cách thay thế các giá trị bị khuyết bằng mean, median hoặc trích xuất một giá trị ngẫu nhiên. Nói cách khác, chúng ta đã học về gán mean/median và gán mẫu ngẫu nhiên. Các phương pháp này giả định rằng dữ liệu bị khuyết hoàn toàn ngẫu nhiên (MCAR).

Có thể sử dụng các phương pháp khác khi các giá trị không bị khuyết ngẫu nhiên, ví dụ: gán giá trị bất kỳ hoặc gán giá trị ở cuối phân phối. Tuy nhiên, các kỹ thuật gán này sẽ ảnh hưởng đáng kể đến phân phối của biến, do đó không phù hợp với các mô hình tuyến tính.

**Chúng ta có thể làm gì nếu dữ liệu không khuyết hoàn toàn ngẫu nhiên mà cần sử dụng mô hình tuyến tính?**

Nếu dữ liệu không bị khuyết ngẫu nhiên, chúng ta có thể thay thế các quan sát bị khuyết bằng mean/median/mode VÀ **gắn cờ** các quan sát bị khuyết đó với một **chỉ số khuyết dữ liệu**. Chỉ số khuyết dữ liệu là một biến nhị phân bổ sung cho biết dữ liệu có bị khuyết cho một quan sát (1) hay không (0).


### Những biến nào có thể thêm chỉ số khuyết dữ liệu?

Chúng ta có thể thêm chỉ số khuyết dữ liệu cho cả biến dạng số và biến hạng mục.

#### Lưu ý

Việc thêm chỉ số khuyết dữ liệu không bao giờ được sử dụng độc lập mà luôn được sử dụng cùng với một kỹ thuật gán khác, có thể là gán mean/median cho các biến dạng số hoặc gán hạng mục thường xuất hiện cho các biến hạng mục. Chúng ta cũng có thể sử dụng gán mẫu ngẫu nhiên cùng với thêm chỉ số khuyết dữ liệu cho cả biến hạng mục và biến dạng số.

Cặp kết hợp sử dụng:

- Gán mean/median + chỉ số khuyết dữ liệu (biến dạng số)
- Gán hạng mục thường xuất hiện + chỉ số khuyết dữ liệu (biến hạng mục)
- Gán mẫu ngẫu nhiên + chỉ số khuyết dữ liệu (biến hạng mục và biến dạng số)

### Giả định
.
- Dữ liệu bị khuyết có tính dự đoán.

### Ưu điểm

- Dễ thực hiện.
- Nắm bắt được tầm quan trọng của "khuyết" nếu có.

### Hạn chế

- Mở rộng không gian đặc trưng.
- Biến ban đầu vẫn cần được gán để loại bỏ NaN.

Việc thêm chỉ số khuyết dữ liệu sẽ làm tăng 1 biến cho mỗi biến trong tập dữ liệu có các giá trị bị khuyết. Vì vậy, nếu tập dữ liệu chứa 10 đặc trưng và tất cả đều có giá trị bị khuyết thì sau khi thêm chỉ số khuyết dữ liệu, chúng ta sẽ có tập dữ liệu với 20 đặc trưng: 10 đặc trưng ban đầu cộng với 10 đặc trưng nhị phân bổ sung, cho biết mỗi biến ban đầu liệu giá trị bị khuyết hay không. Đây có thể không phải là vấn đề với tập dữ liệu có vài chục đến vài trăm biến, nhưng nếu tập dữ liệu ban đầu chứa hàng nghìn biến thì việc tạo một biến bổ sung để chỉ ra NA, chúng ta sẽ có các tập dữ liệu rất lớn.

#### Quan trọng

Ngoài ra, dữ liệu có xu hướng bị khuyết trong cùng một quan sát trên nhiều biến, điều này thường dẫn đến nhiều biến chỉ số khuyết dữ liệu thực sự tương tự hoặc giống hệt nhau.

### Lưu ý cuối cùng

Gán mean/median/mode thường được thực hiện cùng với việc thêm một biến để nắm bắt các quan sát có dữ liệu bị khuyết, dưới 2 góc độ: nếu dữ liệu bị khuyết hoàn toàn ngẫu nhiên, điều này sẽ được dự tính bởi gán mean/median/mode, còn nếu không, nó sẽ được ghi lại bởi chỉ số khuyết dữ liệu.


Cả hai phương pháp đều rất dễ triển khai và nên đây là lựa chọn hàng đầu trong các cuộc thi khoa học dữ liệu. Xem ví dụ về giải pháp giành chiến thắng trong KDD 2009 Cup: ["Winning the KDD Cup Orange Challenge with Ensemble Selection](http://www.mtome.com/Publications/CiML/CiML-v3-book.pdf).


## Trong bản mô phỏng này:

Chúng ta sẽ sử dụng tập dữ liệu Titanic và giá nhà ở Ames.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# chia các tập dữ liệu
from sklearn.model_selection import train_test_split

In [2]:
# load tập dữ liệu Titanic với một số biến để biểu diễn

data = pd.read_csv('./datatset/titanic.csv', usecols=['age', 'fare', 'survived'])
data.head()

,survived,age,fare
0,1,29.0000,211.3375
1,1,0.9167,151.5500
2,0,2.0000,151.5500
3,0,30.0000,151.5500
4,0,25.0000,151.5500


In [3]:
# xem phần trăm NA

data.isnull().mean()

survived    0.000000
age         0.200917
fare        0.000764
dtype: float64

Chúng ta không nhất thiết phải học bất cứ thứ gì từ tập huấn luyện để thêm một chỉ số khuyết dữ liệu nhị phân, vì vậy về nguyên tắc, chúng ta có thể thực hiện điều này trong tập dữ liệu ban đầu, sau đó tách thành tập huấn luyện và tập kiểm tra. Tuy nhiên, tôi không khuyến khích cách thực hiện này. Ngoài ra, nếu sử dụng scikit-learn để thêm chỉ số khuyết dữ liệu, cần tìm hiểu gán các đặc trưng nào từ tập huấn luyện: là các đặc trưng mà biến nhị phân cần được thêm. Chúng ta sẽ xem thêm về các cách triển khai khác của chỉ số khuyết dữ liệu trong notebook khác. Bây giờ, hãy xem cách tạo chỉ số khuyết dữ liệu nhị phân theo cách thủ công.

In [14]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data[['age', 'fare']],  # các yếu tố dự báo
    data['survived'],  # mục tiêu
    test_size=0.3,  # % các quan sát trong tập kiểm tra
    random_state=0)  # seed đảm bảo khả năng tái lặp

X_train.shape, X_test.shape

((916, 2), (393, 2))

In [15]:
# khám phá dữ liệu bị khuyết trong tập huấn luyện
# % sẽ khá giống với %
# của toàn bộ tập dữ liệu

X_train.isnull().mean()

age     0.191048
fare    0.000000
dtype: float64

In [16]:
## Yêu cầu 1: thêm chỉ số khuyết dữ liệu

# thực hiện đơn giản bằng np.where từ numpy
# và isnull từ pandas:

## VIẾT CODE Ở ĐÂY:
X_train['Age_NA'] = np.where(X_train['age'].isnull(), 1, 0)
X_test['Age_NA'] = np.where(X_test['age'].isnull(), 1, 0)

X_train.head()

,age,fare,Age_NA
501,13.0,19.5000,0
588,4.0,23.0000,0
402,30.0,13.8583,0
1193,NaN,7.7250,1
686,22.0,7.7250,0


<details><summary> Gợi ý </summary>

[where()](https://numpy.org/doc/stable/reference/generated/numpy.where.html)

</details>

In [17]:
# mean của biến nhị phân trùng với
# phần trăm các giá trị bị khuyết trong biến ban đầu

X_train['Age_NA'].mean()

0.19104803493449782

In [18]:
# biến ban đầu vẫn hiển thị giá trị bị khuyết
# cần được thay bằng bất kỳ kỹ thuật nào
# chúng ta đã học

X_train.isnull().mean()

age       0.191048
fare      0.000000
Age_NA    0.000000
dtype: float64

In [20]:
def impute_na(df, variable, mean_median):
    ## VIẾT CODE Ở ĐÂY:    
    return  df[variable].fillna(mean_median)

In [19]:
median = X_train['age'].median()
median

28.0

In [21]:
X_train['age'] = impute_na(X_train,'age',median)
X_train

,age,fare,Age_NA
501,13.0000,19.5000,0
588,4.0000,23.0000,0
402,30.0000,13.8583,0
1193,28.0000,7.7250,1
686,22.0000,7.7250,0
...,...,...,...
763,0.1667,20.5750,0
835,28.0000,8.0500,1
1216,28.0000,7.7333,1
559,20.0000,36.7500,0


In [22]:
X_test['age'] = impute_na(X_test,'age',median)
X_test

,age,fare,Age_NA
1139,38.0,7.8958,0
533,21.0,21.0000,0
459,42.0,27.0000,0
1150,28.0,14.5000,1
393,25.0,31.5000,0
...,...,...,...
914,33.0,7.8542,0
580,31.0,21.0000,0
1080,28.0,7.7500,1
1249,28.0,7.7500,1


In [23]:
## Yêu cầu 2: ví dụ về gán median
# kiểm tra xem còn giá trị bị khuyết nào không
X_train.isnull().mean()

age       0.0
fare      0.0
Age_NA    0.0
dtype: float64

### Tập dữ liệu giá nhà

In [24]:
# chúng ta sẽ sử dụng các biến sau
# một số là biến hạng mục, một số là biến dạng số

cols_to_use = [
    'LotFrontage', 'MasVnrArea', # biến số
    'BsmtQual', 'FireplaceQu', # biến hạng mục
    'SalePrice' # target
]

In [25]:
# load tập dữ liệu houseprice

data = pd.read_csv('./datatset/house-price/houseprice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 5)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice
0,65.0,196.0,Gd,NaN,208500
1,80.0,0.0,Gd,TA,181500
2,68.0,162.0,Gd,TA,223500
3,60.0,0.0,TA,Gd,140000
4,84.0,350.0,Gd,TA,250000


In [26]:
# kiểm tra các biến có giá trị khuyết

data.isnull().mean()

LotFrontage    0.177397
MasVnrArea     0.005479
BsmtQual       0.025342
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [27]:
# hãy chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

In [28]:
## Yêu cầu 3: tạo hàm để thêm chỉ số khuyết dữ liệu
# biến nhị phân

def missing_indicator(df, variable):
    ## VIẾT CODE Ở ĐÂY:    
    return np.where(df[variable].isnull(), 1, 0)

In [29]:
## Yêu cầu 4: lặp qua tất cả các biến và thêm một biến nhị phân
# chỉ số khuyết dữ liệu với hàm vừa tạo

for variable in cols_to_use:
    ## VIẾT CODE Ở ĐÂY:
    X_train[variable+'_NA'] = missing_indicator(X_train,variable)
    X_test[variable+'_NA'] = missing_indicator(X_test,variable)
    
X_train.head()

<ipython-input-29-83288cbc17ba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[variable+'_NA'] = missing_indicator(X_train,variable)
<ipython-input-29-83288cbc17ba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[variable+'_NA'] = missing_indicator(X_test,variable)


,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA,SalePrice_NA
64,NaN,573.0,Gd,NaN,219500,1,0,0,1,0
682,NaN,0.0,Gd,Gd,173000,1,0,0,0,0
960,50.0,0.0,TA,NaN,116500,0,0,0,1,0
1384,60.0,0.0,TA,NaN,105000,0,0,0,1,0
1100,60.0,0.0,TA,NaN,60000,0,0,0,1,0


In [ ]:
# đánh giá mean của các chỉ số khuyết dữ liệu

# trước tiên thu thập các biến chỉ số khuyết dữ liệu với một
# list comprehension
missing_ind = [col for col in X_train.columns if 'NA' in col]

# tính mean
X_train[missing_ind].mean()

LotFrontage_NA    0.184932
MasVnrArea_NA     0.004892
BsmtQual_NA       0.023483
FireplaceQu_NA    0.467710
SalePrice_NA      0.000000
dtype: float64

In [30]:
# mean của chỉ số khuyết dữ liệu
# trùng với phần trăm các giá trị bị khuyết
# trong biến ban đầu

X_train.isnull().mean()

LotFrontage       0.184932
MasVnrArea        0.004892
BsmtQual          0.023483
FireplaceQu       0.467710
SalePrice         0.000000
LotFrontage_NA    0.000000
MasVnrArea_NA     0.000000
BsmtQual_NA       0.000000
FireplaceQu_NA    0.000000
SalePrice_NA      0.000000
dtype: float64

In [31]:
## Yêu cầu 5: tạo một hàm để điền các giá trị bị khuyết với một giá trị:
# chúng ta sử dụng một hàm tương tự trong các notebook trước
# nên đã quen thuộc với nó

def impute_na(df, variable, value):
    ## VIẾT CODE Ở ĐÂY:
    return df[variable].fillna(value)

In [32]:
## Yêu cầu 6: gán NA với median cho 
# các biến hạng mục
# chúng ta đã tính median sử dụng tập huấn luyện

median = X_train['LotFrontage'].median()
## VIẾT CODE Ở ĐÂY:
X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
X_test['LotFrontage'] =  impute_na(X_train, 'LotFrontage', median)

median = X_train['MasVnrArea'].median()
## VIẾT CODE Ở ĐÂY:
X_train['MasVnrArea'] = impute_na(X_train, 'MasVnrArea', median)
X_test['MasVnrArea'] = impute_na(X_train, 'MasVnrArea', median)


# gán NA trong các biến hạng mục với
# hạng mục thường xuất hiện nhất (mode)
# mode cần được tìm hiểu từ tập huấn luyện

mode = X_train['BsmtQual'].mode()[0]
## VIẾT CODE Ở ĐÂY:
X_train['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)
X_test['BsmtQual'] = impute_na(X_train, 'BsmtQual', mode)

## VIẾT CODE Ở ĐÂY:
mode = X_train['FireplaceQu'].mode()[0]
X_train['FireplaceQu'] = impute_na(X_train, 'FireplaceQu', mode)
X_test['FireplaceQu'] = impute_na(X_train, 'FireplaceQu', mode)

<ipython-input-32-f743a96975d8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['LotFrontage'] = impute_na(X_train, 'LotFrontage', median)
<ipython-input-32-f743a96975d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LotFrontage'] =  impute_na(X_train, 'LotFrontage', median)
<ipython-input-32-f743a96975d8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [33]:
# kiểm tra xem có còn NA không
X_train.isnull().mean()

LotFrontage       0.0
MasVnrArea        0.0
BsmtQual          0.0
FireplaceQu       0.0
SalePrice         0.0
LotFrontage_NA    0.0
MasVnrArea_NA     0.0
BsmtQual_NA       0.0
FireplaceQu_NA    0.0
SalePrice_NA      0.0
dtype: float64

Chúng ta hiện có gấp đôi số lượng đặc trưng so với tập dữ liệu ban đầu. Tập dữ liệu ban đầu có 4 biến, tập dữ liệu được tiền xử lý chứa 8 biến cộng với mục tiêu.

In [34]:
X_train

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,SalePrice,LotFrontage_NA,MasVnrArea_NA,BsmtQual_NA,FireplaceQu_NA,SalePrice_NA
64,69.0,573.0,Gd,Gd,219500,1,0,0,1,0
682,69.0,0.0,Gd,Gd,173000,1,0,0,0,0
960,50.0,0.0,TA,Gd,116500,0,0,0,1,0
1384,60.0,0.0,TA,Gd,105000,0,0,0,1,0
1100,60.0,0.0,TA,Gd,60000,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
763,82.0,673.0,Gd,Gd,337000,0,0,0,0,0
835,60.0,0.0,Gd,Gd,128000,0,0,0,1,0
1216,68.0,0.0,TA,Gd,112000,0,0,1,1,0
559,69.0,18.0,Gd,TA,234000,1,0,0,0,0
